# Libraries

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors
import pandas as pd
import numpy as np
import sklearn.preprocessing
import sklearn.model_selection
import sklearn.metrics
import xgboost

# Read data

In [13]:
work_dir='/home/ggomarr/Documents/Education/20170623 Udemy - Machine Learning A-Z: Hands-On Python and R in Data Science/' + \
         'Course data/Part 10 - Model Selection & Boosting/Section 49 - XGBoost/'
df=pd.read_csv(work_dir+'Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
X=df[['Geography','Gender','Age','EstimatedSalary','CreditScore',
      'Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember']].values
Y=df['Exited'].values
X[:5],Y[:5]

(array([['France', 'Female', 42, 101348.88, 619, 2, 0.0, 1, 1, 1],
        ['Spain', 'Female', 41, 112542.58, 608, 1, 83807.86, 1, 0, 1],
        ['France', 'Female', 42, 113931.57, 502, 8, 159660.8, 3, 1, 0],
        ['France', 'Female', 39, 93826.63, 699, 1, 0.0, 2, 0, 0],
        ['Spain', 'Female', 43, 79084.1, 850, 2, 125510.82, 1, 1, 1]], dtype=object),
 array([1, 0, 1, 0, 0]))

# Clean up missing data

In [6]:
sum(X!=X),sum(np.isnan(Y))

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 0)

# Create dummy variables

In [7]:
encoder_1=sklearn.preprocessing.LabelEncoder().fit(X[:,0])
X[:,0]=encoder_1.transform(X[:,0])
encoder_2=sklearn.preprocessing.LabelEncoder().fit(X[:,1])
X[:,1]=encoder_2.transform(X[:,1])
one_hot_encoder=sklearn.preprocessing.OneHotEncoder(categorical_features=[0]).fit(X)
X=one_hot_encoder.transform(X).toarray()
X[:5]

array([[  1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   4.20000000e+01,   1.01348880e+05,
          6.19000000e+02,   2.00000000e+00,   0.00000000e+00,
          1.00000000e+00,   1.00000000e+00,   1.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00,
          0.00000000e+00,   4.10000000e+01,   1.12542580e+05,
          6.08000000e+02,   1.00000000e+00,   8.38078600e+04,
          1.00000000e+00,   0.00000000e+00,   1.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   4.20000000e+01,   1.13931570e+05,
          5.02000000e+02,   8.00000000e+00,   1.59660800e+05,
          3.00000000e+00,   1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   3.90000000e+01,   9.38266300e+04,
          6.99000000e+02,   1.00000000e+00,   0.00000000e+00,
          2.00000000e+00,   0.00000000e+00,   0.00000000e+00],
    

# Avoid the dummy variable trap

In [8]:
X=np.delete(X,0,1)
X[:5]

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          4.20000000e+01,   1.01348880e+05,   6.19000000e+02,
          2.00000000e+00,   0.00000000e+00,   1.00000000e+00,
          1.00000000e+00,   1.00000000e+00],
       [  0.00000000e+00,   1.00000000e+00,   0.00000000e+00,
          4.10000000e+01,   1.12542580e+05,   6.08000000e+02,
          1.00000000e+00,   8.38078600e+04,   1.00000000e+00,
          0.00000000e+00,   1.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          4.20000000e+01,   1.13931570e+05,   5.02000000e+02,
          8.00000000e+00,   1.59660800e+05,   3.00000000e+00,
          1.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          3.90000000e+01,   9.38266300e+04,   6.99000000e+02,
          1.00000000e+00,   0.00000000e+00,   2.00000000e+00,
          0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   1.00000000e+00,   0.00000000e+00,
          4.30

# Feature scaling

In [9]:
X_scaled=X
# X_scaler=sklearn.preprocessing.StandardScaler().fit(X)
# X_scaled=X_scaler.transform(X)
# X_scaled[:5]

# Train/test split

In [10]:
X_train,X_test,Y_train,Y_test=sklearn.cross_validation.train_test_split(X_scaled,Y,
                                                                        test_size=0.2,random_state=0)
Y_train.size,Y_test.size

(8000, 2000)

# XGBoost

In [11]:
model_classes={ 'XGBoost':{ 'model':xgboost.XGBClassifier,
                            'params_pre':{},
                            'params_fit':{} },
              }

In [12]:
for class_name in model_classes:
    print('=== {} ==='.format(class_name))
    model_class=model_classes[class_name]['model']
    params_pre=model_classes[class_name]['params_pre']
    params_fit=model_classes[class_name]['params_fit']
    model=model_class(**params_pre).fit(X_train,Y_train,**params_fit)
    accuracies=sklearn.model_selection.cross_val_score(model,X_train,Y_train,
                                                       cv=10)
    print('\n10-fold mean and std:')
    print('{:.2f}, {:.2f}'.format(accuracies.mean(),accuracies.std()))
    print('\nScore:')
    print('Train {:0.2f} - Test {:0.2f}'.format(model.score(X_train,Y_train),model.score(X_test,Y_test)))
    print('\nConfusion matrix:')
    print(sklearn.metrics.confusion_matrix(Y_test,model.predict(X_test)))
    print('\nClassification report:')
    print(sklearn.metrics.classification_report(Y_test,model.predict(X_test)))

=== XGBoost ===

10-fold mean and std:
0.86, 0.01

Score:
Train 0.87 - Test 0.86

Confusion matrix:
[[1521   74]
 [ 197  208]]

Classification report:
             precision    recall  f1-score   support

          0       0.89      0.95      0.92      1595
          1       0.74      0.51      0.61       405

avg / total       0.86      0.86      0.85      2000

